In [6]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import xgboost as xgb

df = pd.DataFrame()

x_labels = ["title", "thumbnail", "tags"]
y_label = "views"
x = df.loc[x_labels]
y = df.loc[y_labels]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror',
                           colsample_bytree = 0.3, 
                           learning_rate = 0.1,
                           max_depth = 5, 
                           alpha = 10, 
                           n_estimators = 10)

xgb_reg.fit(x_train, y_train)
pred = xgb_reg.predict(x_test)

print(np.sqrt(mean_squared_error(y_test, pred)))

KeyError: "None of [Index(['title', 'thumbnail', 'tags'], dtype='object')] are in the [index]"